In [29]:
import numpy as np
import pandas as pd
import altair as alt

In [48]:
np.random.seed(1)

source = pd.DataFrame({
    'x': np.arange(100),
    'A': np.random.randn(100).cumsum(),
    'B': np.random.randn(100).cumsum(),
    'C': np.random.randn(100).cumsum(),
})

source.head()

,x,A,B,C
0,0,1.624345,-0.447129,-0.400878
1,1,1.012589,0.777379,0.423127
2,2,0.484417,1.180871,-0.139178
3,3,-0.588551,1.774449,1.815700
4,4,0.276856,0.679537,0.483748


In [52]:
base = alt.Chart(source).mark_circle(opacity=0.5).transform_fold(
    fold=['A', 'B', 'C'],
    as_=['category', 'y'],
).encode(
    alt.X('x:Q'),
    alt.Y('y:Q'),
    alt.Color('category:N'),
)

base

alt.Chart(...)